# Correlations


In [12]:
%load_ext autoreload
from ExemPy import *
from ExemPy.utils import *
from ExemPy.viz import *
from ExemPy.GCM import *
%aimport ExemPy, ExemPy.utils, ExemPy.viz, ExemPy.GCM
%autoreload 1
import math
import random
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.optimize import minimize
import seaborn as sns
sns.set(style='ticks', context='paper')
colors=["#e3c934","#68c4bf","#c51000","#287271"]
sns.set_palette(colors)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Read in data and confusion matrix
pb52 = pd.read_csv('pb52_data//pb52.csv')
pbcm = pd.read_csv('pb52_data//pbcm.csv').drop([0]).set_index(
    'vowelChoice').rename_axis(None)
pb52 = HzToBark(pb52, ["F0", "F1", "F2", "F3"])
pb52.sample(5)

,type,gender,speaker,vowel,repetition,F0,F1,F2,F3,z0,z1,z2,z3
362,m,m,19,KIT,1,132,370,1750,2700,1.161644,3.727382,12.116226,15.003691
1458,c,f,73,NURSE,1,300,540,1770,2040,3.028850,5.260960,12.192172,13.143100
289,m,m,15,STRUT,2,110,660,960,2450,0.894686,6.223664,8.284247,14.364444
454,m,m,23,FOOT,1,125,390,900,2100,1.077314,3.919319,7.906713,13.337241
417,m,m,21,GOOSE,2,145,290,1000,2300,1.316770,2.925511,8.527432,13.944883


In [14]:
# Set parameters
catslist = ["type", "vowel"]
cval = 25
exemplars = pb52
test = pb52

In [15]:
dims1 = {'z0' : 1,
         'z1' : 3.585,
         'z2' : 2.246,
         'z3' : 2.736}

dims2 = {'z0' : 1,
         'z1' : 2.72,
         'z2' : 1.322,
         'z3' : 0.882}

dims3 = {'z0' : 1,
         'z1' : 1.589,
         'z2' : 0.586,
         'z3' : 0.55}

dims4 = {'z0' : 1,
         'z1' : 2.534,
         'z2' : 1.891,
         'z3' : 1.784}

dims5 = {'z0' : 1,
         'z1' : 1.685,
         'z2' : 1.59,
         'z3' : 1.162}

In [16]:
weightslist = [dims1, dims2, dims3, dims4, dims5]

### Next, let's categorize!
This may take a minute!

In [17]:
# Flatten the confusion matrix for comparison
pbflat = pd.Series(pbcm.stack(), name = "PB")

In [18]:
i = 1
rowlist=[]

print("trial", "rms PB", "r PB", "rms HG", "r HG")
for w in weightslist:
    choices = multicat(
        testset = test,
        cloud = exemplars,
        cats = catslist,
        dimsdict = w,
        c = cval,
        N = 1,
        exclude_self = True)
    
    cm = confusion(choices, catslist)['vowel']
    flat = pd.Series(cm.stack(), name = "GCM")
    matrices = pd.concat([pbflat, flat], axis=1)
    matrices.replace(0, np.nan)
    
    rms = (((matrices.PB-matrices.GCM) ** 2).mean()) ** .5
    r = matrices['PB'].corr(matrices['GCM'])
    
    matrices2 = pd.concat([hgflat, flat], axis=1)
    matrices2.replace(0, np.nan)
    
    rms2 = (((matrices2.HG-matrices2.GCM) ** 2).mean()) ** .5
    r2 = matrices2['HG'].corr(matrices2['GCM'])
    
    print(i, np.round(rms,3), np.round(r,3), np.round(rms2,3), np.round(r2,3))

    i += 1

trial rms PB r PB rms HG r HG
1 0.073 0.964 0.073 0.964
2 0.075 0.964 0.075 0.964
3 0.074 0.966 0.074 0.966
4 0.073 0.965 0.073 0.965
5 0.075 0.966 0.075 0.966
